In [1]:
import torch
import torch.nn.functional as F
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from tqdm import tqdm

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


/home/tahlilmahfuz/RAG_Implementation/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [2]:
# Load ABSA model
absa_model_name = "yangheng/deberta-v3-base-absa-v1.1"
absa_tokenizer = AutoTokenizer.from_pretrained(absa_model_name)
absa_model = AutoModelForSequenceClassification.from_pretrained(absa_model_name).to(device)

/home/tahlilmahfuz/RAG_Implementation/venv/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
# Load overall sentiment model
sentiment_model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_model = pipeline("sentiment-analysis", model=sentiment_model_name,
                           tokenizer=sentiment_model_name, device=0 if torch.cuda.is_available() else -1)

Device set to use cuda:0


In [ ]:
# Define aspects you're interested in analyzing
aspects = ["feature", "ads", "performance"]

# Load your review data
import sqlite3
conn = sqlite3.connect("Auto_Reply_v3.db")
queries = [
    'SELECT "translation" AS Review FROM All_Reviews',
    'SELECT "translation" AS Review FROM "Auto Reply Hadith"',
    'SELECT "translation" AS Review FROM Auto_Reply_Reviews',
    'SELECT "translation" AS Review FROM Auto_Reply_Reviews_Al_Quran'
]
df = pd.concat([pd.read_sql_query(q, conn) for q in queries], ignore_index=True)
conn.close()

In [5]:
# Function to run ABSA for multiple aspects
def get_absa_sentiments(sentence):
    results = {}
    for aspect in aspects:
        # Format input as [CLS] sentence [SEP] aspect [SEP]
        inputs = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect} [SEP]",
                                return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
        with torch.no_grad():
            outputs = absa_model(**inputs)
        probs = F.softmax(outputs.logits, dim=1).detach().cpu().numpy()[0]
        label = ["Negative", "Neutral", "Positive"][probs.argmax()]
        results[aspect] = label
    return results

In [6]:
# Function to get overall sentiment
def get_overall_sentiment(sentence):
    try:
        result = sentiment_model([sentence])[0]
        return result["label"]
    except Exception:
        return "Unknown"

# Apply sentiment analysis
aspect_sentiments = []
overall_sentiments = []

print("Running sentiment analysis...")
for review in tqdm(df["Review"], desc="Processing Reviews"):
    absa = get_absa_sentiments(review)
    aspect_sentiments.append(absa)
    overall_sentiments.append(get_overall_sentiment(review))

# Expand aspect sentiment dictionary into columns
aspect_df = pd.DataFrame(aspect_sentiments)
df = pd.concat([df, aspect_df], axis=1)
df["overall_sentiment"] = overall_sentiments
df

Running sentiment analysis...


Processing Reviews: 100%|██████████| 1000/1000 [00:23<00:00, 42.32it/s]


,Review,feature,ads,performance,overall_sentiment
0,"Alhamdulillah.. the best al quran application,...",Positive,Positive,Positive,positive
1,the best quraan app you can find. it literally...,Positive,Positive,Positive,positive
2,Very helpful,Positive,Positive,Positive,positive
3,The recent edition does explain the source of ...,Neutral,Neutral,Neutral,neutral
4,May Allah reward you richly rewarded,Positive,Neutral,Positive,positive
...,...,...,...,...,...
995,God bless you,Positive,Neutral,Positive,positive
996,It needs update. I experience a lot of issues ...,Negative,Negative,Negative,negative
997,May God bless you and reward you with all the ...,Positive,Positive,Positive,positive
998,Very useful and easy to understand,Positive,Positive,Positive,positive


In [7]:
# Save to CSV
df.to_csv("IslamicApp_Aspect_Sentiment.csv", index=False)
print("Analysis complete! Results saved to IslamicApp_Aspect_Sentiment.csv")

Analysis complete! Results saved to IslamicApp_Aspect_Sentiment.csv
